Based on: 

https://www.tensorflow.org/get_started/mnist/beginners 

https://www.oreilly.com/learning/not-another-mnist-tutorial-with-tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tempfile
import shutil

# print the tensor flow version
print(tf.__version__)

In [ ]:
from utils import display_errors
from utils import unfold_labels
from dataset_helper import DataSetHelper

In [ ]:
num_classes = 10
learning_rate = 0.2
batch_size = 100

In [ ]:
# setup tensorboard
tensorboard_file_name = "./TensorFlowBoard/tensor_flow_basic"
shutil.rmtree(tensorboard_file_name, ignore_errors = True) # start fresh each time
writer = tf.summary.FileWriter(tensorboard_file_name)

In [ ]:
#load mnist data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

training_data_set = DataSetHelper(x_train, y_train, one_hot = True, num_classes = num_classes)
test_data_set = DataSetHelper(x_test, y_test, one_hot = True, num_classes = num_classes)

In [ ]:
training_set_size = x_train.shape[0]
epochs = training_set_size * 3

In [ ]:
# define the graph
# z = W*x + b
# y = softmax(z)
x = tf.placeholder(tf.float32, [None, 784], name = "x")
W = tf.Variable(tf.random.uniform([784, 10]), name = "W")
b = tf.Variable(tf.random.uniform([10]), name = "b")
z = tf.add(tf.matmul(x, W), b, name = "z")
y = tf.nn.softmax(z, name = "y")
y_ = tf.placeholder(tf.float32, [None, 10], name = "y_")

In [ ]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

# calculate the accuracy on the training set
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1), name = "correct_prediction")
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = "accuracy")

prediction = tf.argmax(y,1)

In [ ]:
with tf.Session() as sess:
    # write the graph to the tensorboard log
    writer.add_graph(sess.graph)
    
    tf.global_variables_initializer().run()

    for index in range(epochs):
        batch_xs, batch_ys = training_data_set.next_batch(batch_size)
        
        if index % (epochs/10) == 0:
            train_accuracy = accuracy.eval(feed_dict={x: batch_xs, y_: batch_ys})
            print("Batch:", index, "; Accuracy: ", train_accuracy)
        
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    
    pred = prediction.eval(feed_dict={x: test_data_set.images}, session=sess)
    print("Accuracy: ", sess.run(accuracy, feed_dict={x: test_data_set.images, y_: test_data_set.labels}))

In [ ]:
labels = test_data_set.labels.argmax(axis=1)
conf = tf.confusion_matrix(labels, pred, 10)
with tf.Session() as sess:
    print(conf.eval(session=sess))

In [ ]:
display_errors(x_test, test_data_set.labels, pred, grid = 8)